![image](competi.png)

#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

In [347]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [348]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("data/hospital_train.csv", index_col=0)
print(df.shape)
df.head()

(100000, 17)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,
161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60


In [349]:
df_test = pd.read_csv("data/hospital_test.csv", index_col=0)
print(df_test.shape)
df_test.head()

(133744, 16)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,,,,,,,,,,,,,,,,
314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0


In [350]:
df.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17'],
      dtype='object')

In [351]:
df_columns = pd.read_csv("data/columns_meaning.csv", index_col=0)
df_index = df_columns[:1].values[0][0]
df_columns = df_columns[1:].values.tolist()


In [352]:
df_columns_2 = []
for i in range(len(df_columns)):
    df_columns_2.append(df_columns[i][0])
df_columns_2

['Unique code for the Hospital',
 'Unique code for the type of Hospital',
 'City Code of the Hospital',
 'Region Code of the Hospital',
 'Number of Extra rooms available in the Hospital',
 'Department overlooking the case',
 'Code for the Ward type',
 'Code for the Ward Facility',
 'Condition of Bed in the Ward',
 'Unique Patient Id',
 'City Code for the patient',
 'Admission Type registered by the Hospital',
 'Severity of the illness recorded at the time of admission',
 'Number of Visitors with the patient',
 'Age of the patient',
 'Deposit at the Admission Time',
 'Stay Days by the patient']

In [353]:
df_columns = df_columns_2


In [354]:
df_index

'Case_ID registered in Hospital'

In [355]:
df.columns.values

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17'], dtype=object)

In [356]:
df.columns = df_columns
df.index.name = df_index

In [357]:
df

,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time,Stay Days by the patient
Case_ID registered in Hospital,,,,,,,,,,,,,,,,,
161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237869,12,a,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,More than 100 Days
254763,28,b,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,51-60
69788,6,a,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,31-40


In [358]:
df.dropna(inplace=True)

In [359]:
df

,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time,Stay Days by the patient
Case_ID registered in Hospital,,,,,,,,,,,,,,,,,
161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237869,12,a,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,More than 100 Days
254763,28,b,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,51-60
69788,6,a,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,31-40


## Encoders


In [360]:
print(df["Unique code for the type of Hospital"].unique())
print(df["Region Code of the Hospital"].unique())
print(df["Department overlooking the case"].unique())
print(df["Code for the Ward type"].unique())
print(df["Code for the Ward Facility"].unique())
print(df["Admission Type registered by the Hospital"].unique())
print(df["Severity of the illness recorded at the time of admission"].unique())
print(df["Age of the patient"].unique())
print(df["Stay Days by the patient"].unique())

print("\n", len(df["Deposit at the Admission Time"].unique()))



['a' 'c' 'f' 'b' 'd' 'e' 'g']
['X' 'Z' 'Y']
['gynecology' 'anesthesia' 'radiotherapy' 'TB & Chest disease' 'surgery']
['R' 'Q' 'S' 'P' 'T' 'U']
['F' 'B' 'E' 'C' 'D' 'A']
['Urgent' 'Trauma' 'Emergency']
['Moderate' 'Extreme' 'Minor']
['21-30' '51-60' '71-80' '11-20' '31-40' '0-10' '61-70' '41-50' '81-90'
 '91-100']
['0-10' '21-30' '11-20' '51-60' '31-40' '71-80' 'More than 100 Days'
 '41-50' '81-90' '61-70' '91-100']

 6342


In [361]:
severity_labels = ['Minor', 'Moderate', 'Extreme']
age_labels = ['0-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100']
#days_labels =['0-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100','More than 100 Days']

#days_labels
oe = preprocessing.OrdinalEncoder(categories=[severity_labels, age_labels])

In [362]:
#//, "Stay Days by the patient"// En ambas lineas.
X_encoded = oe.fit_transform(df[["Severity of the illness recorded at the time of admission", "Age of the patient"]])
df[["Severity of the illness recorded at the time of admission", "Age of the patient"]] = X_encoded
df.head()

,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time,Stay Days by the patient
Case_ID registered in Hospital,,,,,,,,,,,,,,,,,
161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,1.0,2,2.0,2817.0,0-10
159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,1.0,4,5.0,4498.0,21-30
309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,1.0,2,7.0,4573.0,11-20
279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,1.0,4,1.0,7202.0,51-60
147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,1.0,2,5.0,3398.0,51-60


In [363]:
le = preprocessing.LabelEncoder()

In [364]:
col1_encoded = le.fit_transform(df["Unique code for the type of Hospital"])
col2_encoded = le.fit_transform(df["Department overlooking the case"])
col3_encoded = le.fit_transform(df["Code for the Ward type"])
col4_encoded = le.fit_transform(df["Code for the Ward Facility"])
col5_encoded = le.fit_transform(df["Admission Type registered by the Hospital"])
col6_encoded = le.fit_transform(df["Region Code of the Hospital"])

df["Unique code for the type of Hospital"] = col1_encoded
df["Department overlooking the case"] = col2_encoded
df["Code for the Ward type"] = col3_encoded
df["Code for the Ward Facility"] = col4_encoded
df["Admission Type registered by the Hospital"] = col5_encoded
df["Region Code of the Hospital"] = col6_encoded
df.head()

,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time,Stay Days by the patient
Case_ID registered in Hospital,,,,,,,,,,,,,,,,,
161528,6,0,6,0,2,2,2,5,4.0,45810,2.0,2,1.0,2,2.0,2817.0,0-10
159472,23,0,6,0,4,2,1,5,2.0,128565,15.0,1,1.0,4,5.0,4498.0,21-30
309765,2,2,5,2,2,1,3,5,3.0,46565,5.0,2,1.0,2,7.0,4573.0,11-20
279614,32,5,9,1,3,2,3,1,4.0,124546,6.0,0,1.0,4,1.0,7202.0,51-60
147791,14,0,1,0,3,2,3,4,2.0,22729,8.0,2,1.0,2,5.0,3398.0,51-60


In [365]:
df.describe()

,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time
count,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000,98484.000000
mean,18.339578,1.254955,4.783894,0.779020,3.195707,1.941787,1.889566,3.290870,2.620446,65955.330551,7.226900,0.781061,0.909346,3.291966,4.129615,4880.696976
std,8.625392,1.529145,3.104605,0.751995,1.169959,0.563335,0.803153,1.685989,0.873974,37872.848150,4.717302,0.689658,0.662146,1.783924,1.889445,1082.590128
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1800.000000
25%,11.000000,0.000000,2.000000,0.000000,2.000000,2.000000,1.000000,2.000000,2.000000,33198.000000,4.000000,0.000000,0.000000,2.000000,3.000000,4189.000000
50%,19.000000,1.000000,5.000000,1.000000,3.000000,2.000000,2.000000,4.000000,3.000000,65912.500000,8.000000,1.000000,1.000000,3.000000,4.000000,4741.000000
75%,26.000000,2.000000,7.000000,1.000000,4.000000,2.000000,2.000000,5.000000,3.000000,98623.250000,8.000000,1.000000,1.000000,4.000000,5.000000,5411.000000
max,32.000000,6.000000,13.000000,2.000000,24.000000,4.000000,5.000000,5.000000,4.000000,131624.000000,38.000000,2.000000,2.000000,32.000000,9.000000,10842.000000


### 1. Definir X e y

In [366]:
X = df.drop(["Stay Days by the patient"],1)
X.shape

(98484, 16)

In [367]:
#PredictorScaler=MinMaxScaler()
#PredictorScalerFit=PredictorScaler.fit(X)
#X=PredictorScalerFit.transform(X)

In [368]:
#skY=df['price'].skew()
#skY
y = df["Stay Days by the patient"]
y.shape

(98484,)

In [369]:
#data_score = pd.DataFrame({'price':np.log(df['price'])})

In [370]:
#sk = data_score['price'].skew()
#sk

In [371]:
#y = np.log(y)

### 2. Dividir X_train, X_test, y_train, y_test

In [372]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [373]:
y_train

Case_ID registered in Hospital
87357      11-20
52780       0-10
123600     11-20
61355      21-30
219572      0-10
           ...  
63226      31-40
171147     21-30
140505    91-100
34711      21-30
208948     11-20
Name: Stay Days by the patient, Length: 78787, dtype: object

### 3. Asignar el modelo (vacío) a una variable
Aquí meteríais los parámetros. 

**Consejo**: Usa GridSearch y vuélvete loca o loco probando modelos e hiperparámetros.

In [374]:
pipe = Pipeline(steps=[
    ('classifier', CatBoostClassifier())
])

params = {
    'classifier': [CatBoostClassifier()],
    'classifier__iterations': [200],
    'classifier__learning_rate':[0.08],
    'classifier__depth':[8],
    'classifier__l2_leaf_reg':[3]
}


#'classifier__n_neighbors=': [11]
model = GridSearchCV(estimator = pipe,
                  param_grid = params,
                  cv = 10,
                  verbose=1,
                  n_jobs=-1)

### 4. Entrenar el modelo con X_train, y_train

In [375]:
model.fit(X_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
0:	learn: 2.2637402	total: 127ms	remaining: 25.3s
1:	learn: 2.1607744	total: 246ms	remaining: 24.3s
2:	learn: 2.0823108	total: 359ms	remaining: 23.6s
3:	learn: 2.0172796	total: 480ms	remaining: 23.5s
4:	learn: 1.9649458	total: 598ms	remaining: 23.3s
5:	learn: 1.9187526	total: 724ms	remaining: 23.4s
6:	learn: 1.8798361	total: 829ms	remaining: 22.9s
7:	learn: 1.8439958	total: 944ms	remaining: 22.6s
8:	learn: 1.8134799	total: 1.06s	remaining: 22.4s
9:	learn: 1.7876287	total: 1.17s	remaining: 22.3s
10:	learn: 1.7640265	total: 1.28s	remaining: 22.1s
11:	learn: 1.7435356	total: 1.4s	remaining: 21.9s
12:	learn: 1.7262066	total: 1.51s	remaining: 21.8s
13:	learn: 1.7081840	total: 1.63s	remaining: 21.6s
14:	learn: 1.6918260	total: 1.74s	remaining: 21.4s
15:	learn: 1.6783088	total: 1.85s	remaining: 21.3s
16:	learn: 1.6659109	total: 1.96s	remaining: 21.1s
17:	learn: 1.6556876	total: 2.08s	remaining: 21s
18:	learn: 1.6453465	total: 2.19s	

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier',
                                        <catboost.core.CatBoostClassifier object at 0x0000028171FD6648>)]),
             n_jobs=-1,
             param_grid={'classifier': [<catboost.core.CatBoostClassifier object at 0x000002817A02CF88>],
                         'classifier__depth': [8],
                         'classifier__iterations': [200],
                         'classifier__l2_leaf_reg': [3],
                         'classifier__learning_rate': [0.08]},
             verbose=1)

In [376]:
print(model.best_estimator_)
print(model.best_score_)
print(model.best_params_)

Pipeline(steps=[('classifier',
                 <catboost.core.CatBoostClassifier object at 0x000002817A173908>)])
0.41768310013658283
{'classifier': <catboost.core.CatBoostClassifier object at 0x000002817A02CF88>, 'classifier__depth': 8, 'classifier__iterations': 200, 'classifier__l2_leaf_reg': 3, 'classifier__learning_rate': 0.08}


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

_Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `diamonds_test.csv` nos arriesgamos a que digamos, no salga lo esperado._

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `diamonds_train.csv` al completo.

**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**

Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `diamonds_test.csv` para predecir.

**¿De dónde saco `diamonds_test.csv`?**

In [377]:
X_pred = df_test
X_pred.columns = X_train.columns
X_pred.head()

,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time
0,,,,,,,,,,,,,,,,
314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0


In [378]:
severity_labels = ['Minor', 'Moderate', 'Extreme']
age_labels = ['0-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100']


oe_2 = preprocessing.OrdinalEncoder(categories=[severity_labels, age_labels])

In [379]:
X_encoded = oe_2.fit_transform(X_pred[["Severity of the illness recorded at the time of admission", "Age of the patient"]])
X_encoded

array([[0., 4.],
       [1., 3.],
       [0., 7.],
       ...,
       [0., 3.],
       [2., 3.],
       [1., 5.]])

In [380]:
X_pred[["Severity of the illness recorded at the time of admission", "Age of the patient"]] = X_encoded
X_pred.head()

,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time
0,,,,,,,,,,,,,,,,
314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,0.0,2,4.0,4778.0
208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,1.0,3,3.0,5734.0
305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,0.0,3,7.0,5064.0
266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,2.0,4,3.0,3254.0
13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,0.0,3,2.0,4639.0


In [381]:
col1_encoded = le.fit_transform(X_pred["Unique code for the type of Hospital"])
col2_encoded = le.fit_transform(X_pred["Department overlooking the case"])
col3_encoded = le.fit_transform(X_pred["Code for the Ward type"])
col4_encoded = le.fit_transform(X_pred["Code for the Ward Facility"])
col5_encoded = le.fit_transform(X_pred["Admission Type registered by the Hospital"])
col6_encoded = le.fit_transform(X_pred["Region Code of the Hospital"])

X_pred["Unique code for the type of Hospital"] = col1_encoded
X_pred["Department overlooking the case"] = col2_encoded
X_pred["Code for the Ward type"] = col3_encoded
X_pred["Code for the Ward Facility"] = col4_encoded
X_pred["Admission Type registered by the Hospital"] = col5_encoded
X_pred["Region Code of the Hospital"] = col6_encoded
X_pred.head()

,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time
0,,,,,,,,,,,,,,,,
314114,19,0,7,1,4,2,3,2,2.0,59278,8.0,0,0.0,2,4.0,4778.0
208989,15,2,5,2,3,2,3,5,2.0,102253,15.0,1,1.0,3,3.0,5734.0
305872,17,4,1,0,4,2,2,4,4.0,5828,4.0,0,0.0,3,7.0,5064.0
266099,3,2,3,2,4,0,2,0,2.0,56642,9.0,2,2.0,4,3.0,3254.0
13228,6,0,6,0,4,2,2,5,1.0,116266,8.0,0,0.0,3,2.0,4639.0


In [382]:
#PredictorScaler=MinMaxScaler()
#PredictorScalerFit=PredictorScaler.fit(X_pred)
#X_pred=PredictorScalerFit.transform(X_pred)

Porque:
    - SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 3 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [383]:
predictions_submit = model.predict(X_pred).reshape(1,-1)[0]
print(len(predictions_submit))
predictions_submit

133744


array(['11-20', '51-60', '11-20', ..., '11-20', '21-30', '21-30'],
      dtype=object)

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

**¿De dónde saco `sample_submission.csv`?**

In [384]:
sample = pd.read_csv("data/sample_submission.csv")

In [385]:
sample.head()

,id,days
0,314114,11-20
1,208989,31-40
2,305872,81-90
3,266099,21-30
4,13228,31-40


In [386]:
sample.shape

(133744, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [387]:
submission = pd.DataFrame({"id": X_pred.index, "days": predictions_submit})

In [388]:
submission.head()

,id,days
0,314114,11-20
1,208989,51-60
2,305872,11-20
3,266099,51-60
4,13228,21-30


In [389]:
submission.shape

(133744, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [390]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission1.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [391]:
help(chequeator)

Help on function chequeator in module __main__:

chequeator(df_to_submit)
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.



In [392]:
chequeator(submission)

You're ready to submit!
